In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import seaborn as sbn



In [ ]:
from matplotlib.pylab import rcParams
rcParams["figure.figsize"] = 10,6


 # Loading of Dataset


In [ ]:
bCancerDf = pd.read_csv("breast-w_csv.csv",na_values=["?"])
bCancerDf.head(10)



In [ ]:
bCancerDf.shape


 # Checking For Null Values

In [ ]:
bCancerDf.isna().sum()


 # Decriptive Statistics of Data For Numeric Columns

In [ ]:
bCancerDf.describe(include=np.number)



In [ ]:
bCancerDf.describe(include=np.object)


 # To Know The Best Option To Fix The Missing Values

In [ ]:
sbn.distplot(bCancerDf["Bare_Nuclei"],bins = 10)


 # Encoding of Target To Numeric Column

In [ ]:
from sklearn.preprocessing import LabelBinarizer



In [ ]:
labeler = LabelBinarizer()



In [ ]:
final_bCancerDf = bCancerDf.iloc[:]



In [ ]:
final_bCancerDf.loc[:,["Class"]] = labeler.fit_transform(final_bCancerDf.loc[:,["Class"]])



In [ ]:
labeler.classes_


 # Fixing Missing Values

In [ ]:
from sklearn.impute import SimpleImputer



In [ ]:
myImputer = SimpleImputer(strategy="median")



In [ ]:
final_bCancerDf.loc[:,["Bare_Nuclei"]] = myImputer.fit_transform(final_bCancerDf.loc[:,["Bare_Nuclei"]])



In [ ]:
final_bCancerDf.isna().sum()



In [ ]:
final_bCancerDf.head()


 # Scaling of Features Excluding The Target to Deal With Euclidean Distance

In [ ]:
from sklearn.preprocessing import RobustScaler



In [ ]:
scaler = RobustScaler()
final_bCancerDf.iloc[:,:-1] = scaler.fit_transform(final_bCancerDf.iloc[:,:-1])


 # Checking For Correlation

In [ ]:
sbn.heatmap(final_bCancerDf.corr(),annot=True,linewidths=.5,robust=True)


 # Preparing Of Data For Traing And Testing

In [ ]:
from sklearn.model_selection import train_test_split



In [ ]:
Xtrain,Xtest,Ytrain,Ytest = train_test_split(final_bCancerDf.iloc[:,:-1].values,final_bCancerDf.iloc[:,-1].values.ravel(),random_state=42)


# Supervised ML

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# Using The GridSearchCv To Find the Best Parameters For Our RandomForestClassifier

In [ ]:
forestClassifier = RandomForestClassifier()

In [ ]:
parameters= [{"n_estimators":[10,15,20],
    "criterion":['entropy'],
    "random_state":[32,42],
    "class_weight":["balanced"]},{"n_estimators":[10,15,20],
    "criterion":['entropy'],
    "random_state":[32,42],
    "class_weight":["balanced_subsample"]}]

In [ ]:
gridSearch = GridSearchCV(estimator=forestClassifier,param_grid=parameters,scoring="accuracy",cv=17,n_jobs=-1)

In [ ]:
gridSearch.fit(Xtrain,y=Ytrain)

In [ ]:
best_score = gridSearch.best_score_
bes_params = gridSearch.best_params_

In [ ]:
best_score

In [ ]:
bes_params

In [ ]:
gridCnn = confusion_matrix(Ytest,gridSearch.predict(Xtest))
gridCnn

# Using Naive Bayes

In [ ]:
naiveClassifier = GaussianNB()

In [ ]:
naiveClassifier.fit(Xtrain,Ytrain)

In [ ]:
naiveClassifier.score(Xtest,Ytest)

In [ ]:
naive_cnn = confusion_matrix(Ytest,naiveClassifier.predict(Xtest))
naive_cnn

 # Setup For Deep Learning Model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.activations import relu,sigmoid
from keras.optimizers import RMSprop



In [ ]:
cancerInitialModel = Sequential()



In [ ]:
cancerInitialModel.add(Dense(50,activation=relu,input_dim=9))
cancerInitialModel.add(Dropout(0.5))
cancerInitialModel.add(BatchNormalization())
cancerInitialModel.add(Dense(100,activation=relu))
cancerInitialModel.add(Dropout(0.5))
cancerInitialModel.add(BatchNormalization())
cancerInitialModel.add(Dense(100,activation=relu))
cancerInitialModel.add(Dropout(0.5))
cancerInitialModel.add(BatchNormalization())
cancerInitialModel.add(Dense(50,activation=relu))
cancerInitialModel.add(Dropout(0.2))
cancerInitialModel.add(BatchNormalization())
cancerInitialModel.add(Dense(1,activation=sigmoid))
cancerInitialModel.summary()



In [ ]:
cancerInitialModel.compile(RMSprop(lr=0.002),loss="binary_crossentropy",metrics=["accuracy"])


 # Running Of Model

In [ ]:
cancerInitialModel.fit(x=Xtrain,y=Ytrain,epochs=100,validation_split=0.2)


 # Checks To See How Well Model Did

In [ ]:
pred_classes = cancerInitialModel.predict_classes(Xtest)



In [ ]:
cnMatrix =confusion_matrix(Ytest,pred_classes)



In [ ]:
cnMatrix



In [ ]:
pred_classes



In [ ]:
cancerInitialModel.history.history.keys()



In [ ]:
validation_loss = np.mean(cancerInitialModel.history.history["val_loss"])
validation_acc = np.mean(cancerInitialModel.history.history["val_acc"])
train_loss = np.mean(cancerInitialModel.history.history["loss"])
train_acc = np.mean(cancerInitialModel.history.history["acc"])



In [ ]:
validation_acc



In [ ]:
validation_loss



In [ ]:
train_acc



In [ ]:
train_loss

